In [1]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from statistics import mean
from numpy.linalg import norm

C:\Users\kevin\anaconda3\envs\chatgpt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Human Prompts

In [2]:
human_prompt_files = [
    "../sentence_embeddings/data/amazonTurkDesPrompt1.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt2.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt3.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt4.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt5.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt6.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt7.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt8.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt9.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt10.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt11.csv",
    "../sentence_embeddings/data/amazonTurkDesPromptNA.csv",
]

human_prompts = []

for human_prompt in human_prompt_files:
    prompts = []
    with open(human_prompt, 'r') as file: #, encoding='Windows-1252') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            prompts.append(row[0])
    prompts = [i.replace('\n',' ') for i in prompts]
    human_prompts.append(prompts)

## Load GPT Base Prompts

In [3]:
gpt_base_prompt_files = [
    "../sentence_embeddings/data/Design_Prompt1.csv",
    "../sentence_embeddings/data/Design_Prompt2.csv",
    "../sentence_embeddings/data/Design_Prompt3.csv",
    "../sentence_embeddings/data/Design_Prompt4.csv",
    "../sentence_embeddings/data/Design_Prompt5.csv",
    "../sentence_embeddings/data/Design_Prompt6.csv",
    "../sentence_embeddings/data/Design_Prompt7.csv",
    "../sentence_embeddings/data/Design_Prompt8.csv",
    "../sentence_embeddings/data/Design_Prompt9.csv",
    "../sentence_embeddings/data/Design_Prompt10.csv",
    "../sentence_embeddings/data/Design_Prompt11.csv",
    "../sentence_embeddings/data/Design_PromptNA.csv",
]

gpt_base_prompts = []

for gpt_base_prompt in gpt_base_prompt_files:
    prompt = pd.read_csv(gpt_base_prompt, sep=",", header=None)
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        mod_statements = lst[i].replace(lst[i][:3], '')
        prompt.append(mod_statements)

    gpt_base_prompts.append(prompt)

## Load GPT Unique Prompts

In [4]:
gpt_unique_files = [
    "../prompt_engineering/data/zero_shot/unique_1.csv",
    "../prompt_engineering/data/zero_shot/unique_2.csv",
    "../prompt_engineering/data/zero_shot/unique_3.csv",
    "../prompt_engineering/data/zero_shot/unique_4.csv",
    "../prompt_engineering/data/zero_shot/unique_5.csv",
    "../prompt_engineering/data/zero_shot/unique_6.csv",
    "../prompt_engineering/data/zero_shot/unique_7.csv",
    "../prompt_engineering/data/zero_shot/unique_8.csv",
    "../prompt_engineering/data/zero_shot/unique_9.csv",
    "../prompt_engineering/data/zero_shot/unique_10.csv",
    "../prompt_engineering/data/zero_shot/unique_11.csv",
    "../prompt_engineering/data/zero_shot/unique_12.csv"
]

gpt_unique_prompts = []

for gpt_unique_prompt in gpt_unique_files:
    try:
        prompt = pd.read_csv(gpt_unique_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_unique_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_unique_prompts.append(prompt)

## Load GPT Diverse Prompts

In [5]:
gpt_files = [
    "../prompt_engineering/data/zero_shot/diverse_1.csv",
    "../prompt_engineering/data/zero_shot/diverse_2.csv",
    "../prompt_engineering/data/zero_shot/diverse_3.csv",
    "../prompt_engineering/data/zero_shot/diverse_4.csv",
    "../prompt_engineering/data/zero_shot/diverse_5.csv",
    "../prompt_engineering/data/zero_shot/diverse_6.csv",
    "../prompt_engineering/data/zero_shot/diverse_7.csv",
    "../prompt_engineering/data/zero_shot/diverse_8.csv",
    "../prompt_engineering/data/zero_shot/diverse_9.csv",
    "../prompt_engineering/data/zero_shot/diverse_10.csv",
    "../prompt_engineering/data/zero_shot/diverse_11.csv",
    "../prompt_engineering/data/zero_shot/diverse_12.csv"
]

gpt_diverse_prompts = []

for gpt_prompt in gpt_files:
    try:
        prompt = pd.read_csv(gpt_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_diverse_prompts.append(prompt)

## Load GPT Novel Prompts

In [6]:
gpt_files = [
    "../prompt_engineering/data/zero_shot/novel_1.csv",
    "../prompt_engineering/data/zero_shot/novel_2.csv",
    "../prompt_engineering/data/zero_shot/novel_3.csv",
    "../prompt_engineering/data/zero_shot/novel_4.csv",
    "../prompt_engineering/data/zero_shot/novel_5.csv",
    "../prompt_engineering/data/zero_shot/novel_6.csv",
    "../prompt_engineering/data/zero_shot/novel_7.csv",
    "../prompt_engineering/data/zero_shot/novel_8.csv",
    "../prompt_engineering/data/zero_shot/novel_9.csv",
    "../prompt_engineering/data/zero_shot/novel_10.csv",
    "../prompt_engineering/data/zero_shot/novel_11.csv",
    "../prompt_engineering/data/zero_shot/novel_12.csv"
]

gpt_novel_prompts = []

for gpt_prompt in gpt_files:
    try:
        prompt = pd.read_csv(gpt_prompt, sep=",")
    except UnicodeDecodeError:
        prompt = pd.read_csv(gpt_prompt, sep=",", encoding='Windows-1252')
    lst = []
    for d in prompt.values:
        lst.append(d[0])
    prompt = []
    for i in range(len(lst)):
        if type(lst[i]) is not float:
            mod_statements = lst[i].replace(lst[i][:3], '')
            prompt.append(mod_statements)

    gpt_novel_prompts.append(prompt)

## Embed data

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
human_embeddings = [model.encode(prompt) for prompt in human_prompts]
gpt_base_embeddings = [model.encode(prompt) for prompt in gpt_base_prompts]
gpt_unique_embeddings = [model.encode(prompt) for prompt in gpt_unique_prompts]
gpt_novel_embeddings = [model.encode(prompt) for prompt in gpt_novel_prompts]
gpt_diverse_embeddings = [model.encode(prompt) for prompt in gpt_diverse_prompts]


# Similarity Metrics
## Nearest Generated Sample
Using cosine distance
Focus on design space coverage

In [9]:
from itertools import combinations


distances_all = []
std_all = []
distances_total = []
for embedding in [gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]:
    distances_prompt = []
    std_prompt = []
    for count in range(len(embedding)):
        distances_row = []
        iterate = list(range(len(np.array(embedding[count]))))
        max_distance = -1e9
        for combo in combinations(iterate, 2):
            # dot product
            distance = np.dot(human_embeddings[count][combo[0],:], embedding[count][combo[1],:])/(norm(human_embeddings[count][combo[0],:])*norm(embedding[count][combo[1],:]))

            if distance > max_distance:
                max_distance = distance
            distances_row.append(max_distance)
        distances_total.append(np.array(distances_row))
        distances_prompt.append(np.mean(distances_row))
        std_prompt.append(np.std(distances_row))
    distances_all.append(distances_prompt)
    std_all.append(std_prompt)

In [10]:
distances_pd = pd.DataFrame(distances_all, index=['base', 'unique', 'novel', 'diverse']).transpose()
distances_pd.style.background_gradient()

,base,unique,novel,diverse
0,0.737221,0.728106,0.787831,0.772523
1,0.627234,0.754285,0.693840,0.804790
2,0.727203,0.759167,0.757869,0.746776
3,0.777565,0.716467,0.786489,0.738319
4,0.796006,0.708150,0.796256,0.842013
5,0.807182,0.804529,0.686086,0.783779
6,0.767732,0.781708,0.716082,0.697302
7,0.704262,0.716625,0.740587,0.869470
8,0.770319,0.780850,0.753955,0.816280
9,0.764971,0.833361,0.839784,0.820882


In [12]:
all_distance = np.array(distances_total)

C:\Users\kevin\AppData\Local\Temp\ipykernel_44872\1301606569.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_distance = np.array(distances_total)


In [13]:
# base soln std + mean
mean_collect = []
stddev_collect = []
collec_num = []
for i in range(12):
    std_dist = np.std(all_distance[i])
    mu_dist = np.mean(all_distance[i])
    std_dist_collec = (all_distance[i].shape[0]-1)*std_dist**2
    stddev_collect.append(std_dist_collec)
    mu_dist_collec = all_distance[i].shape[0]*mu_dist
    mean_collect.append(mu_dist_collec)
    collec_num.append(all_distance[i].shape[0])

total_mu = np.sum(np.array(mean_collect))/np.sum(np.array(collec_num))
total_std = np.sqrt(np.sum(np.array(stddev_collect))/np.sum(np.array(collec_num)))
print('mean ', total_mu)
print('std ', total_std)

mean  0.7475506116946539
std  0.059973298598778


In [14]:
# unique zero-shot soln std + mean
mean_collect = []
stddev_collect = []
collec_num = []
for i in range(12, 24):
    std_dist = np.std(all_distance[i])
    mu_dist = np.mean(all_distance[i])
    std_dist_collec = (all_distance[i].shape[0]-1)*std_dist**2
    stddev_collect.append(std_dist_collec)
    mu_dist_collec = all_distance[i].shape[0]*mu_dist
    mean_collect.append(mu_dist_collec)
    collec_num.append(all_distance[i].shape[0])

total_mu = np.sum(np.array(mean_collect))/np.sum(np.array(collec_num))
total_std = np.sqrt(np.sum(np.array(stddev_collect))/np.sum(np.array(collec_num)))
print('mean ', total_mu)
print('std ', total_std)

mean  0.7461424653648806
std  0.05561045383301105


In [15]:
# novel zero-shot soln std + mean
mean_collect = []
stddev_collect = []
collec_num = []
for i in range(24, 36):
    std_dist = np.std(all_distance[i])
    mu_dist = np.mean(all_distance[i])
    std_dist_collec = (all_distance[i].shape[0]-1)*std_dist**2
    stddev_collect.append(std_dist_collec)
    mu_dist_collec = all_distance[i].shape[0]*mu_dist
    mean_collect.append(mu_dist_collec)
    collec_num.append(all_distance[i].shape[0])

total_mu = np.sum(np.array(mean_collect))/np.sum(np.array(collec_num))
total_std = np.sqrt(np.sum(np.array(stddev_collect))/np.sum(np.array(collec_num)))
print('mean ', total_mu)
print('std ', total_std)

mean  0.7748129103103597
std  0.0691595598915258


In [16]:
# diverse zero-shot soln std + mean
mean_collect = []
stddev_collect = []
collec_num = []
for i in range(36, 48):
    std_dist = np.std(all_distance[i])
    mu_dist = np.mean(all_distance[i])
    std_dist_collec = (all_distance[i].shape[0]-1)*std_dist**2
    stddev_collect.append(std_dist_collec)
    mu_dist_collec = all_distance[i].shape[0]*mu_dist
    mean_collect.append(mu_dist_collec)
    collec_num.append(all_distance[i].shape[0])

total_mu = np.sum(np.array(mean_collect))/np.sum(np.array(collec_num))
total_std = np.sqrt(np.sum(np.array(stddev_collect))/np.sum(np.array(collec_num)))
print('mean ', total_mu)
print('std ', total_std)

mean  0.7787709691772763
std  0.06504224718231634


# Novelty/Diversity Metrics
## Convex hull hypervolume

In [17]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from tqdm import tqdm

In [18]:
pca = PCA(n_components=3)

volumes_all = []
for prompts in tqdm([human_embeddings, gpt_base_embeddings, gpt_unique_embeddings, gpt_novel_embeddings, gpt_diverse_embeddings]):
    volumes_prompts = []
    for prompt in tqdm(prompts, leave=False):
        embeddings = pca.fit_transform(prompt)
        convex_hull = ConvexHull(embeddings)
        volume_prompt = convex_hull.volume
        volumes_prompts.append(volume_prompt)
    volumes_all.append(volumes_prompts)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.91it/s]


In [19]:
convex_hull_volumes_df = pd.DataFrame(volumes_all, index=['human', 'base', 'unique', 'novel', 'diverse']).transpose()
convex_hull_volumes_df.style.background_gradient()

,human,base,unique,novel,diverse
0,0.249209,0.324143,0.331999,0.254749,0.347631
1,0.243630,0.197065,0.326193,0.150178,0.174642
2,0.321649,0.238125,0.274756,0.160631,0.200796
3,0.317053,0.193241,0.346561,0.148948,0.190490
4,0.338390,0.367969,0.341639,0.264245,0.089236
5,0.261036,0.164198,0.197921,0.074321,0.187192
6,0.330278,0.307938,0.206876,0.245315,0.348458
7,0.276590,0.120958,0.094294,0.147923,0.161171
8,0.327088,0.196484,0.151242,0.301653,0.155180
9,0.306566,0.399708,0.175190,0.104635,0.190654


In [20]:
convex_hull_volumes_df

,human,base,unique,novel,diverse
0,0.249209,0.324143,0.331999,0.254749,0.347631
1,0.243630,0.197065,0.326193,0.150178,0.174642
2,0.321649,0.238125,0.274756,0.160631,0.200796
3,0.317053,0.193241,0.346561,0.148948,0.190490
4,0.338390,0.367969,0.341639,0.264245,0.089236
5,0.261036,0.164198,0.197921,0.074321,0.187192
6,0.330278,0.307938,0.206876,0.245315,0.348458
7,0.276590,0.120958,0.094294,0.147923,0.161171
8,0.327088,0.196484,0.151242,0.301653,0.155180
9,0.306566,0.399708,0.175190,0.104635,0.190654


In [21]:
convex_hull_volumes_df.mean(axis=0)

human      0.294032
base       0.259198
unique     0.266242
novel      0.185539
diverse    0.216658
dtype: float64

In [22]:
convex_hull_volumes_df.mean(axis=1)

0     0.301546
1     0.218341
2     0.239191
3     0.239259
4     0.280296
5     0.176934
6     0.287773
7     0.160187
8     0.226330
9     0.235351
10    0.284878
11    0.281919
dtype: float64

In [23]:
convex_hull_volumes_df.std(axis=1)

0     0.046073
1     0.069426
2     0.062672
3     0.086916
4     0.113562
5     0.067658
6     0.059672
7     0.069968
8     0.082791
9     0.117035
10    0.060381
11    0.111368
dtype: float64